In [1]:
import numpy as np
import pandas as pd # for loading data

In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

## Cleaning the test set lyrics and creating One-Hots for train data's genres and just cleaned lyrics

In [6]:
lyrics, word_list = clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
train_genres = one_hot_genres(train_df['Genre'])

## Replace all the lyrics with their respctive vector of word occurrences to be used as the input layer for the neural network

In [8]:
train_lyrics = np.zeros(shape=(len(train_df),len(word_list)))
for index, value in train_df.iterrows():
    train_lyrics[index] = one_hot(str(train_df['Lyrics'][index]), word_list, len(word_list), 0)
print(train_lyrics.shape)
lyrics_size = len(word_list)

# Cleaning no more used variables
del train_df

(10098, 34260)


## Preparing Validation Set

In [9]:
val_df = pd.read_csv("validation.csv")
lyrics, dummy_list = clean_text(val_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    val_df['Lyrics'][index] = text
    index+=1
del lyrics

In [10]:
val_genres = one_hot_genres(val_df['Genre'])

In [11]:
val_lyrics = np.zeros(shape=(len(val_df), lyrics_size))
for index, value in val_df.iterrows():
    val_lyrics[index] = one_hot(str(val_df['Lyrics'][index]), word_list, lyrics_size, 0)
print(val_lyrics.shape)

# Cleaning no more used variables
del val_df

(2142, 34260)


## Cleaning the test set lyrics and creating One-Hots for test data's genres and just cleaned lyrics

In [15]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_list = clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [16]:
test_genres = one_hot_genres(test_df['Genre'])

In [17]:
test_lyrics = np.zeros(shape=(len(test_df), lyrics_size))
for index, value in test_df.iterrows():
    test_lyrics[index] = one_hot(str(test_df['Lyrics'][index]), word_list, lyrics_size, 0)
print(test_lyrics.shape)

# Cleaning no more used variables
del test_df

(2160, 34260)


# Neural Network stuff 
## **Modify and run only the cells below to change the neural network**

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(units=256, input_shape=(lyrics_size,)))
model.add(Activation('relu'))            # activation layer
model.add(Dropout(0.8))
model.add(Dense(256))
model.add(Dense(8))
model.add(Activation('softmax'))         # output class probabilities

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               8770816   
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 2056      
_________________________________________________________________
activation_3 (Activation)    (None, 8)                 0         
Total params: 8,838,664
Trainable params: 8,838,664
Non-trainable params: 0
____________________________________________

In [24]:
from tensorflow.keras.optimizers import SGD, Adam

optimizer = Adam(lr=0.0001) # lr is the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(train_lyrics, train_genres, epochs=10, validation_data=(val_lyrics, val_genres), batch_size=16, shuffle=True);

Train on 10098 samples, validate on 2142 samples
Epoch 1/10
10098/10098 [==============================] - 26s 3ms/sample - loss: 1.8605 - acc: 0.2966 - val_loss: 1.6215 - val_acc: 0.4468
Epoch 2/10
10098/10098 [==============================] - 28s 3ms/sample - loss: 1.4945 - acc: 0.4612 - val_loss: 1.4204 - val_acc: 0.4981
Epoch 3/10
10098/10098 [==============================] - 27s 3ms/sample - loss: 1.2737 - acc: 0.5516 - val_loss: 1.3445 - val_acc: 0.5261
Epoch 4/10
10098/10098 [==============================] - 26s 3ms/sample - loss: 1.1143 - acc: 0.6137 - val_loss: 1.3030 - val_acc: 0.5299
Epoch 5/10
10098/10098 [==============================] - 26s 3ms/sample - loss: 0.9866 - acc: 0.6615 - val_loss: 1.2867 - val_acc: 0.5444
Epoch 6/10
10098/10098 [==============================] - 25s 2ms/sample - loss: 0.8564 - acc: 0.7149 - val_loss: 1.2826 - val_acc: 0.5476
Epoch 7/10
10098/10098 [==============================] - 26s 3ms/sample - loss: 0.7623 - acc: 0.7498 - val_loss: 1.2

## Testing the Neural Network's performace + Confusion Matrix

In [26]:
results = model.evaluate(test_lyrics, test_genres, batch_size=16)

2160/2160 [==============================] - 1s 595us/sample - loss: 1.2927 - acc: 0.5639


In [ ]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict_classes(test_lyrics)
cm=confusion_matrix(one_hot_reverse(test_genres),y_pred)
print(cm)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values